In [8]:
import os
import numpy as np
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.ssl_ import create_urllib3_context
import json
import time

In [9]:
# solve SLLerror and requests error(caused by prox)
CIPHERS = (
    'ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:'
    'DH+HIGH:ECDH+3DES:DH+3DES:RSA+AESGCM:RSA+AES:RSA+HIGH:RSA+3DES:!aNULL:'
    '!eNULL:!MD5'
)

class DESAdapter(HTTPAdapter):
    """
    A TransportAdapter that re-enables 3DES support in Requests.
    """
    def init_poolmanager(self, *args, **kwargs):
        context = create_urllib3_context(ciphers=CIPHERS)
        kwargs['ssl_context'] = context
        return super(DESAdapter, self).init_poolmanager(*args, **kwargs)

    def proxy_manager_for(self, *args, **kwargs):
        context = create_urllib3_context(ciphers=CIPHERS)
        kwargs['ssl_context'] = context
        return super(DESAdapter, self).proxy_manager_for(*args, **kwargs)

s = requests.Session()
s.mount('https://api.rxivist.org', DESAdapter())
headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
# r = s.get('https://some-3des-only-host.com/some-path')

In [10]:
def grepKeywords(key):
    # build a blank df
    outDf = pd.DataFrame(columns=["idx","downloads","category","post_date","title"])
    # the api of bioRxiv
    url = "https://api.rxivist.org/v1/papers?q="+key+"&metric=downloads"
    json_input = s.get(url , headers = headers)
    json_result = json.loads(json_input.text)
    pagesNum = json_result["query"]["final_page"]
    totalNum = json_result["query"]["total_results"]
    
    for i in range(pagesNum+1):
        url_tmp = "https://api.rxivist.org/v1/papers?q="+key+"&metric=downloads&page="+str(i)
        json_input_tmp = s.get(url_tmp , headers = headers)
        json_result_tmp = json.loads(json_input_tmp.text)
        for n in range(20):
            try:
                tmpT = json_result_tmp["results"][n]
                outDf = outDf.append(pd.DataFrame(
                    {"idx":tmpT["id"] , "downloads":tmpT["metric"] ,
                     "category":tmpT["category"] , "post_date":tmpT["first_posted"],
                     "title":tmpT["title"]},index=[0]), ignore_index=True)
            except IndexError:
                break
        
        time.sleep(3)
    
    if totalNum == outDf.shape[0]:
        return(totalNum, outDf)   
    else:
        return("Some errors in "+key+" searching.")

In [11]:
# Final word list
wordStorage = "artificial-intelligence OR machine-learning OR deep-learning OR unsupervised-learning OR supervised-learning OR artificial-neuron OR  transfer-learning OR complex-network OR multitask-learning OR metric-learning OR ensemble-learning OR convolutional-neural-network OR recurrent-neural-network OR autoencoder OR logistic-regression OR support-vector-machine OR k-nearest-neighbors-algorithm OR random-forest OR generative-adversarial-network OR bayesian-network OR decision-tree OR natural-language-processing OR pattern-recognition OR image-search OR fuzzy-logic OR human-computer-interaction OR life-prediction OR feature-extraction OR intelligent-search OR machine-vision OR fuzzy-control OR expert-system OR image-recognition OR neuromotor-computing OR knowledge-reasoning OR genetic-algorithm OR speech-recognition OR perceptron OR wavelet-transform OR face-recognition OR intelligent-robot".lower().replace("-"," ").split(" or ")
### Neural network must be removed.
print(wordStorage)

['artificial intelligence', 'machine learning', 'deep learning', 'unsupervised learning', 'supervised learning', 'artificial neuron', ' transfer learning', 'complex network', 'multitask learning', 'metric learning', 'ensemble learning', 'convolutional neural network', 'recurrent neural network', 'autoencoder', 'logistic regression', 'support vector machine', 'k nearest neighbors algorithm', 'random forest', 'generative adversarial network', 'bayesian network', 'decision tree', 'natural language processing', 'pattern recognition', 'image search', 'fuzzy logic', 'human computer interaction', 'life prediction', 'feature extraction', 'intelligent search', 'machine vision', 'fuzzy control', 'expert system', 'image recognition', 'neuromotor computing', 'knowledge reasoning', 'genetic algorithm', 'speech recognition', 'perceptron', 'wavelet transform', 'face recognition', 'intelligent robot']


In [12]:
# outDf = pd.DataFrame({"idx":"idx","downloads":"downloads","category":"category",
#                       "post_date":"post_date","title":"title"},index=[0])
outDf = pd.DataFrame(columns=["idx","downloads","category","post_date","title"])
count = 0
for word in wordStorage:
    tmpDf = grepKeywords(word)
    count += tmpDf[0]
    outDf = pd.concat([outDf , tmpDf[1]])
    

In [13]:
print("total number of papers: "+str(count))
# outDf = outDf.drop(0)
if count == outDf.shape[0]:
    print("Duplicate processing:")
    outDf1 = outDf.drop_duplicates(["idx"])
    outDf2 = outDf1.reset_index(drop=True) 
    outDf2.to_csv("./bioRxiv_AIrelated_papers_Final.tsv" , sep="\t" , index=0)
else:
    print("Some errors here, but the origin matrix is output.")
    outDf.to_csv("./bioRxiv_onlyConcat_AIrelated_papers.tsv")

total number of papers: 5033
Duplicate processing:
